In [26]:
import numpy as np

In [27]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [28]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [31]:
n_chars = len(raw_text)
n_vocab = len(chars)
print "Total Characters: ", n_chars
print "Total Vocab: ", n_vocab

Total Characters:  163780
Total Vocab:  58


In [32]:
print char_to_int

{'\n': 0, '!': 2, ' ': 1, '#': 4, '"': 3, '%': 6, '$': 5, "'": 7, ')': 9, '(': 8, '*': 10, '-': 12, ',': 11, '/': 14, '.': 13, '1': 16, '0': 15, '3': 18, '2': 17, '5': 20, '4': 19, '7': 22, '6': 21, '9': 24, '8': 23, ';': 26, ':': 25, '?': 27, '@': 28, '[': 29, ']': 30, '_': 31, 'a': 32, 'c': 34, 'b': 33, 'e': 36, 'd': 35, 'g': 38, 'f': 37, 'i': 40, 'h': 39, 'k': 42, 'j': 41, 'm': 44, 'l': 43, 'o': 46, 'n': 45, 'q': 48, 'p': 47, 's': 50, 'r': 49, 'u': 52, 't': 51, 'w': 54, 'v': 53, 'y': 56, 'x': 55, 'z': 57}


In [33]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns

Total Patterns:  163680


In [39]:
print raw_text[0:10+seq_length]

project gutenberg's alice's adventures in wonderland, by lewis carroll

this ebook is for the use of anyone an


In [1]:
import numpy as np
from neon import logger as neon_logger
from neon.data import Shakespeare
from neon.initializers import Uniform
from neon.layers import GeneralizedCost, LSTM, Affine
from neon.models import Model
from neon.optimizers import RMSProp
from neon.transforms import Logistic, Tanh, Softmax, CrossEntropyMulti
from neon.callbacks.callbacks import Callbacks
from neon.util.argparser import NeonArgparser

In [14]:
from neon.backends import gen_backend
default_overrides = dict(save_path='rnn_text_gen.pickle',
                         serialize=1,
                         batch_size=64)
be = gen_backend(backend='cpu',batch_size=10)

In [15]:
time_steps = 64
hidden_size = 512
gradient_clip_value = 5

In [28]:
path = '/Users/tgn/nervana/data/shakespeare_input_train.txt'

In [29]:
from neon.data import Text

In [44]:
data = Text(20, path)
print data.vocab

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
init = Uniform(low=-0.08, high=0.08)

# model initialization
layers = [
    LSTM(hidden_size, init, activation=Logistic(), gate_activation=Tanh()),
    Affine(len(data.vocab), init, bias=init, activation=Softmax())
]
model = Model(layers=layers)

cost = GeneralizedCost(costfunc=CrossEntropyMulti(usebits=True))

optimizer = RMSProp()

# configure callbacks
callbacks = Callbacks(model, eval_set=data)

# fit and validate
model.fit(data, optimizer=optimizer,
          num_epochs=64, cost=cost, callbacks=callbacks)

Epoch 0   [Train |                    |  461/20580 batches, 3.16 cost, 43.75s]